# Start with the merged File to preprocess it

In [1]:
import pandas as pd
import numpy as np
import ast
import re

In [2]:
merged_df = pd.read_excel('D:/Only_GDB_Operation/Premanual FIles/merged_IGR_20_21_22.xlsx') # Your merged Premanual file here

In [3]:
# Drop the columns which are not necessary
merged_df.drop(columns=['Unnamed: 0','village','sro_code','Remaining_Assumed_BUA','builtup_sqmt','saleable_sqmt', 'project_clean'], inplace=True)

In [4]:
merged_df.shape

(478421, 18)

# Calculate Rate/Sqft by cleaning columns like Kshetrafal and Mobadla

In [ ]:
merged_df

In [5]:
#import static variables from the static file
from static import result_dict, word_number_dict

In [6]:
from utils import clean_mobadla, calculate_rate_sqft

In [7]:
merged_df.rename(columns={'date': 'Transaction Date', 'unit_clean': 'Unit No', 'floor_clean': 'Floor No', 
                            'mobadla': 'Agreement Price(INR)', 'Kshetrafal': 'Net Carpet Area sqmt (KHESTRAFAL)', 'doc_type': 'Transaction Type', 'bhumapan': 'Bhumapan',
                            'Property_Type': 'Property Type','village_Name_Eng': 'IGR Village', 'sro_name': 'SRO Name', 'doc_num': 'Document No', 'bajarbhav': 'Bajarbhav'}, inplace=True)

In [8]:
merged_df['Net Carpet Area sqmt (KHESTRAFAL)'] = clean_mobadla(merged_df['Net Carpet Area sqmt (KHESTRAFAL)'])

In [ ]:
merged_df

In [9]:
merged_df['Rate/sqft(INR)'] = calculate_rate_sqft(merged_df['Agreement Price(INR)'], merged_df['Net Carpet Area sqmt (KHESTRAFAL)'])

# Convert Doc_Type and Floor to English

In [10]:
merged_df['Transaction Type'] = merged_df['Transaction Type'].map(result_dict.get)

In [11]:
merged_df['Floor No'] = merged_df['Floor No'].map(word_number_dict)

In [12]:
# Operation for Unit number column
merged_df['Unit No'] = merged_df['Unit No'].str.extract(r'(\d+)', expand=False).astype(float).astype(pd.Int64Dtype(), errors='ignore')
        

mask = ~merged_df['Unit No'].isnull() & ~merged_df['Property Type'].isnull()


merged_df['Unit No'] = np.where(mask, 
                                    merged_df['Property Type'].astype(str) + ' no. ' + merged_df['Unit No'].astype(str), 
                                    np.nan)

In [13]:
merged_df.to_excel('Merged_File_Preprocessed_Premanual_for_main_file.xlsx', index=False)
#This output file will be the input file for the main.py file